In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [9]:
url = "https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day28-column-transformer/covid_toy.csv"
df = pd.read_csv(url)
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [10]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
#we want to do diffrent diffrent encoding in diffrent columns
#age is column is perfect
#for gender, city columns we want to do OneHotEncoding
#for fever column simple imputer
#for cough column OrdinalEncoding

In [12]:
#  1.There are two ways we can give each encoder separately
#  2.otherwise we can use column transfer

In [22]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=.2)

# 1. Encode each columns separately

In [28]:
#Adding simple imputer in fever column
si=SimpleImputer()
x_train_fever=si.fit_transform(x_train[['fever']])
x_test_fever=si.fit_transform(x_test[['fever']])
x_train_fever.shape,x_test_fever.shape

((80, 1), (20, 1))

In [32]:
#Adding Ordinal Encode in cough column
oe=OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough=oe.fit_transform(x_train[['cough']])
x_test_cough=oe.fit_transform(x_test[['cough']])
x_train_cough.shape,x_test_cough.shape

((80, 1), (20, 1))

In [53]:
#Adding OneHotEncoder in gender and city columns
ohe=OneHotEncoder(drop='first',sparse=False)
x_train_gender_city=ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city=ohe.fit_transform(x_test[['gender','city']])
x_train_gender_city.shape,x_test_gender_city.shape

((80, 4), (20, 4))

In [54]:
#extracting age 
x_train_age=x_train.drop(columns=['gender','fever','cough','city']).values
x_test_age=x_test.drop(columns=['gender','fever','cough','city']).values
x_train_age.shape,x_test_age.shape

((80, 1), (20, 1))

In [55]:
x_train_transformed=np.concatenate((x_train_age,x_train_cough,x_train_fever,x_train_gender_city),axis=1)
x_test_transformed=np.concatenate((x_test_age,x_test_cough,x_test_fever,x_test_gender_city),axis=1)
x_train_transformed.shape,x_train_transformed.shape

((80, 7), (80, 7))

# 2.Encode with column transfer

In [56]:
from sklearn.compose import ColumnTransformer

In [66]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=.2)

In [74]:
transformr=ColumnTransformer(transformers=[('trn1',SimpleImputer(),['fever']),
                                          ('trn2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
                                         ('trn3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])],
                            remainder='passthrough')


In [78]:
transformr.transform(x_train).shape,transformr.transform(x_test).shape

((80, 7), (20, 7))